<H3>PRI 2022: first project delivery</H3>

**GROUP X**
- name, number
- name, number
- name, number

<H3>Part I: demo of facilities</H3>

**A) Indexing facilities**

*A.1 Preprocessing options: statistics*

In [18]:
#rare, most common terms


avg sentences 7.893483146067416
avg characters 2232.4188764044943


*A.2 Indexing statistics*

In [ ]:
#code, statistics and/or charts here
#time, memory

**B) Ranking facilities**

In [19]:
# load files
from mainFunctions.ranking import ranking
from operator import attrgetter
from helper.documentHelper import read_files
from mainFunctions.indexing import indexing
order_ranked = True
text_processing = True
documents = read_files(text_processing, ["business"])

corpus_index = indexing(list(map(attrgetter('text_terms'), documents)))
corpus_idfs: {str: float} = {}

for v in corpus_index:
    corpus_idfs[v] = corpus_index[v].inverted_document_frequency

document = documents[0]

summary1 = ranking(document, 7, None, order_ranked, corpus_idfs, {"rank_option": "tf", "mmr": True})
summary2 = ranking(document, 3, None, order_ranked, corpus_idfs, {"rank_option": "tf-idf", "mmr": False})
summary3 = ranking(document, 3, None, order_ranked, corpus_idfs, {"rank_option": "bm25", "mmr": False})
summary4 = ranking(document, 3, None, order_ranked, corpus_idfs, {"rank_option": "rrf", "mmr": False})
summary5 = ranking(document, 3, None, order_ranked, corpus_idfs, {"rank_option": "tf-idf", "mmr": True})

document.summary = summary5

document_sentences = document.text_sentences
summary_sentences = document.summary
reference_summary_sentences = document.referenceSummary

*B.1 Summarization solution: results for a given document*

In [ ]:
text = document

*B.2 Classic IR models: differences*

In [ ]:
#code, statistics and/or charts here

*B.3 Reciprocal rank funsion*

In [ ]:
#code, statistics and/or charts here

*B.4 Maximal Marginal Relevance*

In [ ]:
#code, statistics and/or charts here

**C) Sentence higlighting facilities**

In [ ]:
#code, statistics and/or charts here

**D) Evaluation facilities**

*D.1 Evaluation options*

In [ ]:
#code, statistics and/or charts here

*D.2 Comparison of settings (IR models, preprocessing)*

In [ ]:
#code, statistics and/or charts here

<H3>Part II: questions materials (optional)</H3>

**(a)** Corpus and summary description. Distribution of informative terms before and after text processing.

In [ ]:
#code, statistics and/or charts here

from src.helper.documentHelper import read_files


documents_no_preprocessing = read_files(False)
documents_preprocessing = read_files(True)

print(documents_preprocessing[0].text)  #Test

# Calculate distribution of informative termsstics and/or charts here

**(b)** Summarization performance for the overall and category-conditional corpora.

In [ ]:
#code, statistics and/or charts here

**...** (additional questions with empirical results)

<H3>Part III: Other caluclations to support the project</H3>

In [20]:
# get average sentences and characters of the reference summaries
from helper.documentHelper import read_files
totalSentences = 0
totalCharacters = 0
totalDocuments = 0
documents = read_files(False)

for document in documents:
    sentences = document.referenceSummary
    totalSentences += len(sentences)
    totalCharacters += len(" ".join(sentences))
    totalDocuments +=1

print("avg sentences", totalSentences / totalDocuments)
print("avg characters", totalCharacters /totalDocuments)

avg sentences 7.893483146067416
avg characters 1008.9977528089887


<H3>END</H3>